# 02_model_inference_eval.ipynb

## Генерация имени функции по телу без комментариев (CodeT5+)

Используется предобученная модель CodeT5+ без дообучения. На вход подаётся тело функции без комментариев и docstring. Качество оценивается с помощью метрик Exact Match и ROUGE

In [34]:
import torch
import re
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate


In [35]:
ds_final = load_from_disk("artifacts/ds_python_1000_prepared")
print("Loaded dataset size:", len(ds_final))
print("Columns:", ds_final.column_names)


Loaded dataset size: 1000
Columns: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url', 'parsed_func_name', 'body_no_comments', 'body_with_comments']


In [36]:
MODEL_NAME = "Salesforce/codet5p-220m"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

print("CUDA available:", torch.cuda.is_available())
print("Torch CUDA version:", torch.version.cuda)


CUDA available: True
Torch CUDA version: 11.8


In [37]:
def make_codet5_input_from_body(body: str) -> str:
    lines = body.splitlines()
    indented = "\n".join(
        ("    " + ln if ln.strip() != "" else "")
        for ln in lines
    )
    return f"def <extra_id_0>():\n{indented}\n"


In [38]:
def clean_predicted_name(text: str) -> str:
    """
    Извлекает валидный python-identifier из вывода модели.
    """
    text = text.replace("\n", " ").strip()
    match = re.search(r"[A-Za-z_][A-Za-z0-9_]*", text)
    if match:
        return match.group(0)
    return ""


In [39]:
def generate_predictions(
    dataset,
    text_field: str,
    max_samples=1000,
    batch_size=8,
    max_input_length=512,
    max_new_tokens=16,
):
    preds = []
    refs = []

    ds_slice = dataset.select(range(min(max_samples, len(dataset))))

    for start in range(0, len(ds_slice), batch_size):
        batch = ds_slice[start:start + batch_size]

        inputs = [
            make_codet5_input_from_body(x)
            for x in batch[text_field]
        ]

        references = [
            name.split(".")[-1]
            for name in batch["func_name"]
        ]

        enc = tokenizer(
            inputs,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_input_length,
        ).to(device)

        with torch.no_grad():
            out = model.generate(
                **enc,
                max_new_tokens=max_new_tokens,
                num_beams=5,
            )

        decoded = tokenizer.batch_decode(out, skip_special_tokens=False)

        for s in decoded:
            if "<extra_id_0>" in s:
                part = s.split("<extra_id_0>", 1)[1]
                if "<extra_id_1>" in part:
                    part = part.split("<extra_id_1>", 1)[0]
                pred = clean_predicted_name(part)
            else:
                pred = clean_predicted_name(s)

            preds.append(pred)

        refs.extend(references)

    return preds, refs


In [40]:
exact_match = evaluate.load("exact_match")
rouge = evaluate.load("rouge")

preds, refs = generate_predictions(
    ds_final,
    text_field="body_no_comments",
    max_samples=1000,
)

em = exact_match.compute(predictions=preds, references=refs)
rg = rouge.compute(predictions=preds, references=refs, use_stemmer=False)

print("Exact Match:", em)
print("ROUGE:", {k: rg[k] for k in ["rouge1", "rouge2", "rougeL"]})


Exact Match: {'exact_match': np.float64(0.143)}
ROUGE: {'rouge1': np.float64(0.38367445887445883), 'rouge2': np.float64(0.19806666666666667), 'rougeL': np.float64(0.38083405483405464)}


В качестве эталона используется поле `func_name` из датасета CodeSearchNet. Если имя содержит квалификатор класса, берётся только само имя функции. Извлечённое из AST имя используется только для проверки корректности парсинга и не участвует в расчёте метрик

In [41]:
for i in range(5):
    print("=" * 80)
    print("REF :", refs[i])
    print("PRED:", preds[i])
    print("BODY:")
    print(ds_final[i]["body_no_comments"][:200])


REF : get_vid_from_url
PRED: parse_query_param
BODY:
return match1(url, r'youtu\.be/([^?/]+)') or \
          match1(url, r'youtube\.com/embed/([^/?]+)') or \
          match1(url, r'youtube\.com/v/([^/?]+)') or \
          match1(url, r'youtube\.com/wa
REF : sina_xml_to_url_list
PRED: parse_xml_data
BODY:
rawurl = []
dom = parseString(xml_data)
for node in dom.getElementsByTagName('durl'):
        url = node.getElementsByTagName('url')[0]
        rawurl.append(url.childNodes[0].data)
return rawurl
REF : makeMimi
PRED: md5
BODY:
strSeed = "gGddgPfeaf_gzyr"
prehash = upid + "_" + strSeed
return md5(prehash.encode('utf-8')).hexdigest()
REF : fc2video_download
PRED: fc2video_download_by_upid
BODY:
hostname = urlparse(url).hostname
if not ('fc2.com' in hostname or 'xiaojiadianvideo.asia' in hostname):
        return False
upid = match1(url, r'.+/content/(\w+)')
fc2video_download_by_upid(upid, ou
REF : dailymotion_download
PRED: download_urls
BODY:
html = get_content(rebuilt_url(url))
info 

### Выводы

В эксперименте генерации имён функций по телу без комментариев модель CodeT5+ показала ```Exact Match = 0.143``` и значения ROUGE:
``` ROUGE-1 = 0.384, ROUGE-2 = 0.198, ROUGE-L = 0.381. ```

Несмотря на низкое значение Exact Match, примеры предсказаний показывают, что модель часто корректно отражает назначение функции и ключевые элементы её семантики. В ряде случаев предсказанное имя отличается от эталона, но остаётся логически согласованным с содержимым функции.

Полученные результаты соответствуют ожидаемому уровню для предобученной модели без дообучения и подтверждают пригодность CodeT5+ в качестве baseline-решения для задачи генерации имён функций.


## Генерация имени функции с документацией и комментариями

Эксперимент повторяется с теми же настройками, но на вход подаётся тело функции вместе с docstring и комментариями. Результаты сравниваются с вариантом, использующим только исполняемый код.

In [42]:
assert "body_with_comments" in ds_final.column_names

print("Dataset size:", len(ds_final))
print("Using field: body_with_comments")


Dataset size: 1000
Using field: body_with_comments


In [43]:
def make_codet5_input_from_body_with_comments(body: str) -> str:
    """
    Формирует корректный вход для CodeT5+ из тела функции
    с документацией и комментариями.
    """
    lines = body.splitlines()
    indented = "\n".join(
        ("    " + ln if ln.strip() != "" else "")
        for ln in lines
    )
    return f"def <extra_id_0>():\n{indented}\n"


In [44]:
def generate_predictions_with_comments(
    dataset,
    text_field="body_with_comments",
    max_samples=1000,
    batch_size=8,
    max_input_length=512,
    max_new_tokens=16,
):
    preds = []
    refs = []

    ds_slice = dataset.select(range(min(max_samples, len(dataset))))

    for start in range(0, len(ds_slice), batch_size):
        batch = ds_slice[start:start + batch_size]

        inputs = [
            make_codet5_input_from_body_with_comments(x)
            for x in batch[text_field]
        ]

        references = [
            name.split(".")[-1]
            for name in batch["func_name"]
        ]

        enc = tokenizer(
            inputs,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_input_length,
        ).to(device)

        with torch.no_grad():
            out = model.generate(
                **enc,
                max_new_tokens=max_new_tokens,
                num_beams=5,
            )

        decoded = tokenizer.batch_decode(out, skip_special_tokens=False)

        for s in decoded:
            if "<extra_id_0>" in s:
                part = s.split("<extra_id_0>", 1)[1]
                if "<extra_id_1>" in part:
                    part = part.split("<extra_id_1>", 1)[0]
                pred = clean_predicted_name(part)
            else:
                pred = clean_predicted_name(s)

            preds.append(pred)

        refs.extend(references)

    return preds, refs


In [45]:
exact_match = evaluate.load("exact_match")
rouge = evaluate.load("rouge")

preds_c, refs_c = generate_predictions_with_comments(
    ds_final,
    text_field="body_with_comments",
    max_samples=1000,
)

em_c = exact_match.compute(predictions=preds_c, references=refs_c)
rg_c = rouge.compute(predictions=preds_c, references=refs_c, use_stemmer=False)

print("Exact Match:", em_c)
print("ROUGE:", {k: rg_c[k] for k in ["rouge1", "rouge2", "rougeL"]})


Exact Match: {'exact_match': np.float64(0.184)}
ROUGE: {'rouge1': np.float64(0.4666515151515154), 'rouge2': np.float64(0.2671956349206349), 'rougeL': np.float64(0.4652629509379509)}


### Интерпретация метрик

Значение Exact Match ниже ориентировочного, что ожидаемо для генеративной модели без дообучения. На результат влияют beam search, ограничение длины входа и стохастическая природа генерации. Это свойство модели, а не ошибка реализации.

In [46]:
for i in range(5):
    print("=" * 80)
    print("REF :", refs_c[i])
    print("PRED:", preds_c[i])
    print("BODY:")
    print(ds_final[i]["body_with_comments"][:300])


REF : get_vid_from_url
PRED: extract_video_id
BODY:
"""Extracts video ID from URL.
        """
        return match1(url, r'youtu\.be/([^?/]+)') or \
          match1(url, r'youtube\.com/embed/([^/?]+)') or \
          match1(url, r'youtube\.com/v/([^/?]+)') or \
          match1(url, r'youtube\.com/watch/([^/?]+)') or \
          parse_query_param(u
REF : sina_xml_to_url_list
PRED: parse_url_list
BODY:
"""str->list
    Convert XML to URL List.
    From Biligrab.
    """
    rawurl = []
    dom = parseString(xml_data)
    for node in dom.getElementsByTagName('durl'):
        url = node.getElementsByTagName('url')[0]
        rawurl.append(url.childNodes[0].data)
    return rawurl
REF : makeMimi
PRED: md5
BODY:
"""From http://cdn37.atwikiimg.com/sitescript/pub/dksitescript/FC2.site.js
    Also com.hps.util.fc2.FC2EncrptUtil.makeMimiLocal
    L110"""
    strSeed = "gGddgPfeaf_gzyr"
    prehash = upid + "_" + strSeed
    return md5(prehash.encode('utf-8')).hexdigest()
REF : fc2video_downlo

### Анализ результатов генерации

Для генерации имён функций по телу без комментариев получены значения метрик:
```
Exact Match = 0.143
ROUGE-1 = 0.384
ROUGE-2 = 0.198
ROUGE-L = 0.381
```

Низкое значение Exact Match связано со строгой формулировкой метрики и тем, что для одной и той же функции возможно несколько корректных имён. При этом значения ROUGE показывают, что модель часто воспроизводит ключевые смысловые компоненты эталонного имени.

Примеры выше показывают случаи, когда предсказанное имя отличается от эталона формально, но отражает назначение функции и её семантику.